In [1]:
import os  # isort:skip # fmt:skip # noqa # nopep8
import sys  # isort:skip # fmt:skip # noqa # nopep8
from pathlib import Path  # isort:skip # fmt:skip # noqa # nopep8

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

sys.path.append(code_dir)
# %load_ext autoreload
# %autoreload 2


In [2]:
from setup_module.imports import * # isort:skip # fmt:skip # noqa # nopep8
from supervised_estimators_get_pipe import *  # isort:skip # fmt:skip # noqa # nopep8


Using MPS


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

0it [00:00, ?it/s]

0it [00:00, ?it/s]

Using MPS


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

0it [00:00, ?it/s]

0it [00:00, ?it/s]

### Set variables

In [3]:
# Variables
warnings.filterwarnings('always')
# Sklearn variables
method = 'Supervised'
results_save_path = f'{models_save_path}{method} Results/'
searchcv_xy_save_path = f'{results_save_path}SearchCV+Xy/'
t = time.time()
n_jobs = -1
n_splits = 10
n_repeats = 3
random_state = 42
refit = True
class_weight = 'balanced'
cv = RepeatedStratifiedKFold(
    n_splits=n_splits, n_repeats=n_repeats, random_state=random_state
)
scoring = 'recall'
scores = [
    'recall', 'accuracy', 'f1', 'roc_auc',
    'explained_variance', 'matthews_corrcoef'
]
scorers = {
    'precision_score': make_scorer(precision_score, zero_division=0),
    'recall_score': make_scorer(recall_score, zero_division=0),
    'accuracy_score': make_scorer(accuracy_score, zero_division=0),
}
analysis_columns = ['Warmth', 'Competence']
text_col = 'Job Description spacy_sentencized'
metrics_dict = {
    'Train - Mean Cross Validation Score': np.nan,
    f'Train - Mean Cross Validation - {scoring.title()}': np.nan,
    f'Train - Mean Explained Variance - {scoring.title()}': np.nan,
    'Test - Mean Cross Validation Score': np.nan,
    f'Test - Mean Cross Validation - {scoring.title()}': np.nan,
    f'Test - Mean Explained Variance - {scoring.title()}': np.nan,
    'Explained Variance': np.nan,
    'Accuracy': np.nan,
    'Balanced Accuracy': np.nan,
    'Precision': np.nan,
    'Average Precision': np.nan,
    'Recall': np.nan,
    'F1-score': np.nan,
    'Matthews Correlation Coefficient': np.nan,
    'Fowlkes–Mallows Index': np.nan,
    'ROC': np.nan,
    'AUC': np.nan,
    f'{scoring.title()} Best Threshold': np.nan,
    f'{scoring.title()} Best Score': np.nan,
    'Log Loss/Cross Entropy': np.nan,
    'Cohen’s Kappa': np.nan,
    'Geometric Mean': np.nan,
    'Classification Report': np.nan,
    'Imbalanced Classification Report': np.nan,
    'Confusion Matrix': np.nan,
    'Normalized Confusion Matrix': np.nan,
}

# Transformer variables
max_length = 512
returned_tensor = 'pt'
cpu_counts = torch.multiprocessing.cpu_count()
device = torch.device('mps') if torch.has_mps and torch.backends.mps.is_built() and torch.backends.mps.is_available(
) else torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device_name = str(device.type)
print(f'Using {device_name.upper()}')
# Set random seed
random.seed(random_state)
np.random.seed(random_state)
torch.manual_seed(random_state)
DetectorFactory.seed = random_state
cores = multiprocessing.cpu_count()
bert_model_name = 'bert-base-uncased'
bert_tokenizer = BertTokenizerFast.from_pretrained(
    bert_model_name, strip_accents=True
)
bert_model = BertForSequenceClassification.from_pretrained(
    bert_model_name
).to(device)

# Plotting variables
pp = pprint.PrettyPrinter(indent=4)
tqdm.tqdm.pandas(desc='progress-bar')
tqdm_auto.tqdm.pandas(desc='progress-bar')
tqdm.notebook.tqdm().pandas(desc='progress-bar')
tqdm_auto.notebook_tqdm().pandas(desc='progress-bar')
# pbar = progressbar.ProgressBar(maxval=10)
mpl.use('MacOSX')
mpl.style.use(f'{code_dir}/setup_module/apa.mplstyle-main/apa.mplstyle')
mpl.rcParams['text.usetex'] = True
font = {'family': 'arial', 'weight': 'normal', 'size': 10}
mpl.rc('font', **font)
plt.style.use('tableau-colorblind10')
plt.set_cmap('Blues')
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 5000)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 3)
pd.set_option('display.float_format', '{:.2f}'.format)


Using MPS


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

0it [00:00, ?it/s]

0it [00:00, ?it/s]

# Functions

In [4]:
def show_and_close_plots():
    plt.show()
    plt.clf()
    plt.cla()
    plt.close()


In [5]:
def close_plots():
    plt.clf()
    plt.cla()
    plt.close()


In [6]:
def make_df_metrics(vectorizers_pipe, classifiers_pipe, metrics_list, analysis_columns=analysis_columns):

    index = pd.MultiIndex.from_product(
        [list(map(lambda classifier: classifier, classifiers_pipe.keys()))],
        names=['Classifiers'],
    )
    columns = pd.MultiIndex.from_product(
        [
            analysis_columns,
            list(map(lambda vectorizer: vectorizer, vectorizers_pipe.keys())),
            metrics_list,
        ],
        names=['Variable', 'Vectorizer', 'Measures'],
    )
    return pd.DataFrame(index=index, columns=columns)


In [7]:
def get_existing_files(
    results_save_path= results_save_path,
    col_names_list=None,
    vectorizer_names_list=None,
    classifier_names_list=None,
):
    if col_names_list is None:
        col_names_list = []
    if vectorizer_names_list is None:
        vectorizer_names_list = []
    if classifier_names_list is None:
        classifier_names_list = []

    print(f'Searching for existing estimators in directory:\n{results_save_path}')

    for estimators_file in glob.glob(f'{results_save_path}*.pkl'):
        col_names_list.append(
            col := estimators_file.split(f'{method} Estimator - ')[-1].split(' - ')[0]
        )
        vectorizer_names_list.append(
            vectorizer_name := estimators_file.split(f'{col} - ')[-1].split(' + ')[0]
        )
        classifier_names_list.append(
            classifier_name := estimators_file.split(f'{vectorizer_name} + ')[-1].split(' (Save_protocol=')[0]
        )

    estimator_names_list = [
        f'{col} - {vectorizer_name} + {classifier_name}'
        for col, vectorizer_name, classifier_name in tqdm_product(
            list(set(col_names_list)),
            list(set(vectorizer_names_list)),
            list(set(classifier_names_list)),
        )
    ]
    return (
        list(set(col_names_list)),
        list(set(vectorizer_names_list)),
        list(set(classifier_names_list)),
        list(set(estimator_names_list))
    )


In [8]:
def class_weights_print_Xy(
    X_train, y_train,
    X_test, y_test,
):
    # Check for consistent length
    check_consistent_length(X_train, y_train)
    check_consistent_length(X_test, y_test)

    # Get train class weights
    train_class_weights = compute_class_weight(class_weight = class_weight, classes = np.unique(y_train), y = y_train)
    train_class_weights_ratio = train_class_weights[0]/train_class_weights[1]
    train_class_weights_dict = dict(zip(np.unique(y_train), train_class_weights))

    # Get train class weights
    test_class_weights = compute_class_weight(class_weight = class_weight, classes = np.unique(y_train), y = y_test)
    test_class_weights_ratio = test_class_weights[0]/test_class_weights[1]
    test_class_weights_dict = dict(zip(np.unique(y_test), test_class_weights))

    print('Done splitting data into training and testing sets.')
    print('='*20)
    print(f'Training set shape: {y_train.shape}')
    print('-'*10)
    print(f'Training set example:\n{X_train[0]}')
    print('~'*10)
    print(f'Testing set shape: {y_test.shape}')
    print('-'*10)
    print(f'Testing set example:\n{X_test[0]}')
    print('~'*10)
    print(f'Training data class weights:\nRatio = {train_class_weights_ratio:.2f} (0 = {train_class_weights[0]:.2f}, 1 = {train_class_weights[1]:.2f})')
    print('-'*10)
    print(f'Testing data class weights:\nRatio = {test_class_weights_ratio:.2f} (0 = {test_class_weights[0]:.2f}, 1 = {test_class_weights[1]:.2f})')
    print('='*20)

    return (
        train_class_weights, train_class_weights_ratio, train_class_weights_dict,
        test_class_weights_dict, test_class_weights_ratio, test_class_weights_dict
    )

In [9]:
# Function to place Xy and CV data in df and save
def load_Xy_search_cv_estimator(
    col, vectorizer_name, classifier_name, protocol,
    results_save_path=results_save_path,
    searchcv_xy_save_path=searchcv_xy_save_path, method=method,
    compression=None, saved_files_list=None,
    path_suffix=None, data_dict=None,
):
    if compression is None:
        compression = False
    if path_suffix is None:
        path_suffix = f' - {str(col)} - {vectorizer_name} + {classifier_name} (Save_protocol={protocol}).pkl'
    if data_dict is None:
        data_dict = {}
    if saved_files_list is None:
        saved_files_list = [
            'Grid Search', 'SearchCV', 'df_cv_results', 'df_train_data', 'df_test_data', 'Estimator'
        ]

    # Load data into dict
    for file_path in glob.glob(f'{searchcv_xy_save_path}*{path_suffix}'):
        file_name = file_path.split(f'{searchcv_xy_save_path}{method} ')[-1].split(path_suffix)[0]

        if 'df_' in file_name:
            data_dict[file_name] = pd.read_pickle(file_path)
        else:
            with open(file_path, 'rb') as f:
                data_dict[file_name] = joblib.load(f)

    # Load estimator
    with open(
        f'{results_save_path}{method} Estimator{path_suffix}', 'rb'
    ) as f:
        data_dict['Estimator'] = joblib.load(f)

    # Train data
    df_train_data = data_dict['df_train_data']
    X_train = df_train_data['X_train'].values
    y_train = df_train_data['y_train'].values
    y_train_pred = df_train_data['y_train_pred'].values
    # Test data
    df_test_data = data_dict['df_test_data']
    X_test = df_test_data['X_test'].values
    y_test = df_test_data['y_test'].values
    y_test_pred = df_test_data['y_test_pred'].values
    y_test_pred_prob = df_test_data['y_test_pred_prob'].values

    # Assign data to variables
    grid_search = data_dict['Grid Search']
    searchcv = data_dict['SearchCV']
    df_cv_results = data_dict['df_cv_results']
    estimator = data_dict['Estimator']
    if 'df_feature_importances' in data_dict.keys():
        saved_files_list.append('df_feature_importances')
        df_feature_importances = data_dict['df_feature_importances']
    else:
        df_feature_importances = None

    # Check data
    check_consistent_length(X_train, y_train, y_train_pred)
    check_consistent_length(X_test, y_test, y_test_pred, y_test_pred_prob)
    check_is_fitted(estimator)

    assert set(data_dict.keys()) == set(saved_files_list), f'Not all files were loaded! Missing: {set(data_dict.keys()) ^ set(saved_files_list)}'
    print(f'Done loading Xy, CV data, and estimator!\n{list(data_dict.keys())}')
    print('='*20)

    # Get class weights and print info
    (
        train_class_weights, train_class_weights_ratio, train_class_weights_dict,
        test_class_weights_dict, test_class_weights_ratio, test_class_weights_dict
    ) = class_weights_print_Xy(
        X_train, y_train,
        X_test, y_test,
    )

    return (
        grid_search, searchcv,
        X_train, y_train, y_train_pred,
        X_test, y_test, y_test_pred, y_test_pred_prob,
        df_feature_importances, estimator,
    )


In [10]:
def compute_metrics_with_estimator(
    estimator, X_test, y_test, col, vectorizer_name, classifier_name,
    cv=cv, return_train_score=None,
):
    if return_train_score is None:
        return_train_score = True

    # Using estimator
    # Cross Validation
    print('-'*20)
    print('Cross Validating without scoring.')
    cv_score_noscoring = cross_validate(
        estimator,
        X_test,
        y_test,
        cv=cv,
        scoring=None,
        return_train_score=True,
    )

    # Cross Validation with scoring
    print('-'*20)
    print(f'Cross Validating with {scores} scoring.')
    cv_score_recall = cross_validate(
        estimator,
        X_test,
        y_test,
        cv=cv,
        scoring=scores,
        return_train_score=True,
    )

    # Get mean and std of cross validation scores
    print('-'*20)
    print('Getting mean and std of cross validation scores.')
    cv_train_scores = cv_score_noscoring['train_score'].mean()
    cv_test_scores = cv_score_noscoring['test_score'].mean()
    cv_train_recall = cv_score_recall['train_recall'].mean()
    cv_test_recall = cv_score_recall['test_recall'].mean()
    cv_train_explained_variance_recall = cv_score_recall['train_explained_variance'].mean()
    cv_test_explained_variance_recall = cv_score_recall['test_explained_variance'].mean()

    # Save cross validation scores to dataframe
    print('-'*20)
    print('Saving cross validation scores to dataframe.')
    df_cv_score_noscoring = pd.DataFrame(cv_score_noscoring)
    df_cv_score_noscoring.to_pickle(f'{df_save_dir}df_cv_score_noscoring - {col}_{vectorizer_name}_{classifier_name}.pkl')
    df_cv_score_recall = pd.DataFrame(cv_score_recall)
    df_cv_score_recall.to_pickle(f'{df_save_dir}df_cv_score_recall - {col}_{vectorizer_name}_{classifier_name}.pkl')

    return (
        df_cv_score_recall,
        cv_train_scores, cv_test_scores,
        cv_train_recall, cv_test_recall,
        cv_train_explained_variance_recall, cv_test_explained_variance_recall
    )


In [11]:
def plot_metrics_with_estimator(
    estimator, X_test, y_test, col, vectorizer_name, classifier_name, random_state=random_state, n_jobs=n_jobs, cv=cv,
    params=None, axis=None, alpha=None, verbose=None
):
    if axis is None:
        axis = 1
    if alpha is None:
        alpha = 0.1
    if verbose is None:
        verbose=1

    # Make param names and values
    if params is None:
        params = {
            param_name: classifier_params
            for classifier_name, classifier_and_params in classifiers_pipe.items()
            if estimator[-1].__class__.__name__ == classifier_name
            for param_name_, classifier_params in classifier_and_params[-1].items()
            for param_name in [param_name_.split(f'{classifier_name}__')[-1]]
            if param_name != 'random_state' and all(isinstance(n, (list, int, float)) for n in classifier_params) and not all(isinstance(n, (bool)) for n in classifier_params)
        }


    # Learning Curves
    print('Plotting Learning Curve.')
    print('-'*20)
    train_sizes, train_scores, test_scores = learning_curve(
        estimator=estimator,
        X=X_train,
        y=y_train,
        cv=cv,
        n_jobs=n_jobs,
        random_state=random_state,
        shuffle=True,
        scoring=scoring,
        verbose=verbose,
        # train_sizes=np.linspace(0.1, 1.0, 10),
    )
    train_scores_mean = np.mean(train_scores, axis=axis)
    train_scores_std = np.std(train_scores, axis=axis)
    test_scores_mean = np.mean(test_scores, axis=axis)
    test_scores_std = np.std(test_scores, axis=axis)

    close_plots()
    plt.figure()
    plt.suptitle(
        f'{str(col)} - Learning Curves for {scoring.title()} - {vectorizer_name} + {classifier_name}'
        )
    plt.xlabel('Training examples')
    plt.ylabel('Score')
    plt.grid()
    plt.fill_between(
        train_sizes, train_scores_mean - train_scores_std, train_scores_mean + train_scores_std, alpha=alpha, color='r'
    )
    plt.fill_between(
        train_sizes, test_scores_mean - test_scores_std, test_scores_mean + test_scores_std, alpha=alpha, color='g'
    )
    plt.plot(
        train_sizes, train_scores_mean, 'o-', color='r', label='Training score'
    )
    plt.plot(
        train_sizes, test_scores_mean, 'o-', color='g', label='Cross-validation score'
    )
    plt.legend(loc='best')

    # Save figure
    for image_save_format in ['eps', 'png', 'svg']:
        plt.savefig(f'{plot_save_path}{method} {str(col)} - Learning Curve - {vectorizer_name} + {classifier_name}.{image_save_format}', format=image_save_format)
    show_and_close_plots()

    # Validation Curve
    for param_name, param_range in params.items():
        param_title = ' '.join(param_name.split('_')).title()
        print(f'Plotting Validation Curve for {param_title}.')
        print('-'*20)
        train_scores, test_scores = validation_curve(
            estimator=estimator[-1],
            X=X_train,
            y=y_train,
            param_name=param_name,
            param_range=param_range,
            cv=cv,
            n_jobs=n_jobs,
            scoring=scorers['recall_score'],
            verbose=1,
        )
        train_scores_mean = np.mean(train_scores, axis=axis)
        train_scores_std = np.std(train_scores, axis=axis)
        test_scores_mean = np.mean(test_scores, axis=axis)
        test_scores_std = np.std(test_scores, axis=axis)

        # Ploting
        plt.figure()
        plt.suptitle(
            f'{str(col)} - Validation Curve for {scoring.title()} - {str(col)} - {vectorizer_name} + {classifier_name}'
        )
        plt.xlabel(param_name)
        plt.ylabel('Score')
        plt.grid()
        plt.fill_between(
            param_range, train_scores_mean - train_scores_std, train_scores_mean + train_scores_std, alpha=alpha, color='r'
        )
        plt.fill_between(
            param_range, test_scores_mean - test_scores_std, test_scores_mean + test_scores_std, alpha=alpha, color='g'
        )
        plt.semilogx(
            param_range, train_scores_mean, label='Training score', color='r'
        )
        plt.semilogx(
            param_range, test_scores_mean, label='Cross-validation score', color='g'
        )
        plt.plot(
            param_range, train_scores_mean, 'o-', color='r', label='Training score'
        )
        plt.plot(
            param_range, test_scores_mean, 'o-', color='g', label='Cross-validation score'
        )
        plt.legend(loc='best')
        show_and_close_plots()

        # Save figure
        for image_save_format in ['eps', 'png', 'svg']:
            plt.savefig(
                f'{plot_save_path}{method} {str(col)} - Validation Curve for {scoring.title()} - {str(col)} - {vectorizer_name} + {classifier_name}.{image_save_format}',
                format=image_save_format
            )


In [12]:
def compute_metrics_with_y_pred(
    y_test, y_test_pred,
    pos_label=None, labels=None, zero_division=None, alpha=None
):
    if pos_label is None:
        pos_label = 1
    if labels is None:
        labels = np.unique(y_test_pred)
    if zero_division is None:
        zero_division = 0
    if alpha is None:
        alpha = 0.1

    # Using y_pred
    explained_variance = metrics.explained_variance_score(y_test, y_test_pred)
    accuracy = metrics.accuracy_score(y_test, y_test_pred)
    balanced_accuracy = metrics.balanced_accuracy_score(y_test, y_test_pred)
    precision = metrics.precision_score(y_test, y_test_pred, pos_label=pos_label, labels=labels, zero_division=zero_division)
    recall = metrics.recall_score(y_test, y_test_pred, pos_label=pos_label, labels=labels, zero_division=zero_division)
    f1 = metrics.f1_score(y_test, y_test_pred, pos_label=pos_label, labels=labels, zero_division=zero_division)
    mcc = metrics.matthews_corrcoef(y_test, y_test_pred)
    fm = metrics.fowlkes_mallows_score(y_test, y_test_pred)
    kappa = metrics.cohen_kappa_score(y_test, y_test_pred, labels=labels)
    gmean_iba = imblearn.metrics.make_index_balanced_accuracy(alpha=alpha, squared=True)(geometric_mean_score)
    gmean = gmean_iba(y_test, y_test_pred)
    report = metrics.classification_report(y_test, y_test_pred, labels=labels, zero_division=zero_division)
    imblearn_report = classification_report_imbalanced(y_test, y_test_pred, labels=labels, zero_division=zero_division)
    cm = metrics.confusion_matrix(y_test, y_test_pred, labels=labels)
    cm_normalized = metrics.confusion_matrix(y_test, y_test_pred, normalize='true', labels=labels)

    return (
        explained_variance, accuracy, balanced_accuracy, precision,
        recall, f1, mcc, fm, kappa, gmean, report, imblearn_report, cm, cm_normalized
    )


In [13]:
def plot_metrics_with_y_pred(
    y_test, y_test_pred, col, vectorizer_name, classifier_name,
    pos_label=None, labels=None
):
    if pos_label is None:
        pos_label = 1
    if labels is None:
        labels = np.unique(y_test_pred)

    # Displays
    close_plots()
    cm_curve = metrics.ConfusionMatrixDisplay.from_predictions(
        y_test, y_test_pred, display_labels=labels, cmap=plt.cm.Blues
    )
    cm_normalized_curve = metrics.ConfusionMatrixDisplay.from_predictions(
        y_test, y_test_pred, normalize='true', display_labels=labels, cmap=plt.cm.Blues
    )
    roc_curve = metrics.RocCurveDisplay.from_predictions(
        y_test, y_test_pred, pos_label=pos_label
    )
    pr_curve = metrics.PrecisionRecallDisplay.from_predictions(
        y_test, y_test_pred, pos_label=pos_label
    )
    calibration_curve = CalibrationDisplay.from_predictions(
        y_test, y_test_pred, pos_label=pos_label
    )
    show_and_close_plots()

    # Plots
    plots_dict = {
        'Confusion Matrix': cm_curve,
        'Normalized Confusion Matrix': cm_normalized_curve,
        'ROC Curve': roc_curve,
        'Precision-Recall Curve': pr_curve,
        'Calibration Curve': calibration_curve,
    }

    print('=' * 20)
    close_plots()
    print('Plotting metrics with y_pred_prob:')
    print('='*20)

    for plot_name, plot_ in plots_dict.items():
        close_plots()
        print(f'Plotting {plot_name}:')
        fig, ax = plt.subplots()
        ax.set_title(
            f'{str(col)} - {plot_name} - {vectorizer_name} + {classifier_name}'
            )
        if plot_name == 'ROC Curve':
            ax.plot([0, 1], [0, 1], 'r--', lw=1)
        try:
            plot_.plot(ax=ax, cmap=plt.cm.Blues)
        except Exception:
            plot_.plot(ax=ax)
        print('=' * 20)

        # Save Plots
        print(f'Saving {plot_name}...')
        for image_save_format in ['eps', 'png', 'svg']:
            plt.savefig(
                f'{plot_save_path}{method} {str(col)} - {plot_name} - {vectorizer_name} + {classifier_name}.{image_save_format}',
                format=image_save_format, dpi=3000, bbox_inches='tight'
            )
        show_and_close_plots()
        print(f'Saved {plot_name}!')
        print('=' * 20)

    # Visualisation with plot_metric
    bc = plot_metric.functions.BinaryClassification(y_test, y_test_pred, labels=[0, 1])

    # Figures
    close_plots()
    fig = plt.figure(figsize=(15, 10))
    plt.subplot2grid((2, 6), (1, 1), colspan=2)
    bc.plot_confusion_matrix(colorbar=True)
    plt.subplot2grid((2, 6), (1, 3), colspan=2)
    bc.plot_confusion_matrix(normalize=True, colorbar=True)
    plt.subplot2grid(shape=(2, 6), loc=(0, 0), colspan=2)
    bc.plot_roc_curve()
    plt.subplot2grid((2, 6), (0, 2), colspan=2)
    bc.plot_precision_recall_curve()
    plt.subplot2grid((2, 6), (0, 4), colspan=2)
    bc.plot_class_distribution()
    bc.print_report()
    for image_save_format in ['eps', 'png', 'svg']:
        plt.savefig(
            f'{plot_save_path}{method} {str(col)} - plot_metric Curves - {vectorizer_name} + {classifier_name}.{image_save_format}',
            format=image_save_format,
            dpi=3000, bbox_inches='tight'
        )
    show_and_close_plots()

    # Heatmap
    print('Plotting Heatmap:')
    close_plots()
    classifications_dict = defaultdict(int)
    for _y_test, _y_test_pred in zip(y_test, y_test_pred):
        if _y_test != _y_test_pred:
            classifications_dict[(_y_test, _y_test_pred)] += 1

    dicts_to_plot = [
        {
            f'True {str(col)} value': _y_test,
            f'Predicted {str(col)} value': _y_test_pred,
            'Number of Classifications': _count,
        }
        for (_y_test, _y_test_pred), _count in classifications_dict.items()
    ]
    df_to_plot = pd.DataFrame(dicts_to_plot)
    df_wide = df_to_plot.pivot_table(
        index=f'True {str(col)} value', 
        columns=f'Predicted {str(col)} value', 
        values='Number of Classifications'
    )
    plt.figure(figsize=(9,7))
    sns.set(style='ticks', font_scale=1.2)
    sns.heatmap(df_wide, linewidths=1, cmap=plt.cm.Blues, annot=True)    
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.suptitle(f'{str(col)} Heatmap - {vectorizer_name} + {classifier_name}')
    print('Saving Heatmap...')
    for image_save_format in ['eps', 'png', 'svg']:
        plt.savefig(
            f'{plot_save_path}{method} {str(col)} - Heatmap - {vectorizer_name} + {classifier_name}.{image_save_format}',
            format=image_save_format,
            dpi=3000, bbox_inches='tight'
        )
    print('Saved Heatmap!')
    show_and_close_plots()



In [14]:
def compute_metrics_with_y_pred_prob(
    y_test, y_test_pred_prob,
    pos_label=None
):
    if pos_label is None:
        pos_label = 1

    average_precision = metrics.average_precision_score(y_test, y_test_pred_prob)
    roc_auc = metrics.roc_auc_score(y_test, y_test_pred_prob)
    fpr, tpr, threshold = metrics.roc_curve(y_test, y_test_pred_prob, pos_label=1)
    auc = metrics.auc(fpr, tpr)
    loss = metrics.log_loss(y_test, y_test_pred_prob)
    precision_pr, recall_pr, threshold_pr = metrics.precision_recall_curve(y_test, y_test_pred_prob, pos_label=1)

    return (
        average_precision, roc_auc, auc,
        fpr, tpr, threshold,loss,
        precision_pr, recall_pr, threshold_pr
    )


In [15]:
def compute_metrics(
    estimator, X_test, y_test, y_test_pred, y_test_pred_prob,
    col, vectorizer_name, classifier_name, cv=cv, n_jobs=n_jobs,
    with_estimator=None, with_y_pred=None, with_y_pred_prob=None,
    pos_label=None, verbose=None,
):
    if pos_label is None:
        pos_label = 1
    if verbose is None:
        verbose = 1
    if with_estimator is None:
        with_estimator = True
    if with_y_pred is None:
        with_y_pred = True
    if with_y_pred_prob is None:
        with_y_pred_prob = True

    # Get metrics
    print('='*20)
    # Using estimator
    if with_estimator:
        print('Computing metrics using estimator.')
        (
            df_cv_score_recall,
            cv_train_scores, cv_test_scores,
            cv_train_recall, cv_test_recall,
            cv_train_explained_variance_recall, cv_test_explained_variance_recall
        ) = compute_metrics_with_estimator(
             estimator, X_test, y_test, col, vectorizer_name, classifier_name,
        )
    # Using y_test_pred
    if with_y_pred:
        print('-'*20)
        print('Computing metrics using y_test_pred.')
        (
            explained_variance, accuracy, balanced_accuracy, precision,
            recall, f1, mcc, fm, kappa, gmean, report, imblearn_report, cm, cm_normalized
        ) = compute_metrics_with_y_pred(
            y_test, y_test_pred
        )
    # Using y_test_pred_prob
    if with_y_pred_prob:
        print('-'*20)
        print('Computing metrics using y_test_pred_prob.')
        (
            average_precision, roc_auc, auc,
            fpr, tpr, threshold,loss,
            precision_pr, recall_pr, threshold_pr
        ) = compute_metrics_with_y_pred_prob(
            y_test, y_test_pred_prob
        )

    #Place metrics into dict
    print('-'*20)
    print('Appending metrics to dict.')
    metrics_dict = {
        'Train - Mean Cross Validation Score': float(cv_train_scores),
        f'Train - Mean Cross Validation - {scoring.title()}': float(cv_train_recall),
        f'Train - Mean Explained Variance - {scoring.title()}': float(cv_train_explained_variance_recall),
        'Test - Mean Cross Validation Score': float(cv_test_scores),
        f'Test - Mean Cross Validation - {scoring.title()}': float(cv_test_recall),
        f'Test - Mean Explained Variance - {scoring.title()}': float(cv_test_explained_variance_recall),
        'Explained Variance': float(explained_variance),
        'Accuracy': float(accuracy),
        'Balanced Accuracy': float(balanced_accuracy),
        'Precision': float(precision),
        'Average Precision': float(average_precision),
        'Recall': float(recall),
        'F1-score': float(f1),
        'Matthews Correlation Coefficient': float(mcc),
        'Fowlkes–Mallows Index': float(fm),
        'ROC': float(roc_auc),
        'AUC': float(auc),
        f'{scoring.title()} Best Threshold': threshold,
        f'{scoring.title()} Best Score': float(best_train_score),
        'Log Loss/Cross Entropy': float(loss),
        'Cohen’s Kappa': float(kappa),
        'Geometric Mean': float(gmean),
        'Classification Report': report,
        'Imbalanced Classification Report': imblearn_report,
        'Confusion Matrix': cm,
        'Normalized Confusion Matrix': cm_normalized
    }

    return (
        metrics_dict, df_cv_score_recall,
        cv_train_scores, cv_test_scores,
        cv_train_recall, cv_test_recall,
        cv_train_explained_variance_recall, cv_test_explained_variance_recall,
        explained_variance, accuracy, balanced_accuracy, precision, recall,
        f1, mcc, fm, kappa, gmean, report, cm, cm_normalized,
        average_precision, roc_auc, auc, fpr, tpr, threshold, 
        loss, precision_pr, recall_pr, threshold_pr,
    )


In [16]:
def plot_metrics(
    estimator, X_test, y_test, y_test_pred, y_test_pred_prob,
    col, vectorizer_name, classifier_name, 
    with_estimator=None, with_y_pred=None, with_y_pred_prob=None
):
    if with_estimator is None:
        with_estimator = True
    if with_y_pred is None:
        with_y_pred = True
    if with_y_pred_prob is None:
        with_y_pred_prob = True

    # Plotting
    print('~'*20)
    print('Plotting metrics.')
    print('~'*20)
    # Using estimator
    if with_estimator:
        plot_metrics_with_estimator(
             estimator, X_test, y_test, col, vectorizer_name, classifier_name,
        )
    # Using y_test_pred
    if with_y_pred:
        plot_metrics_with_y_pred(
            y_test, y_test_pred, col, vectorizer_name, classifier_name,
        )
    print('='*20)


In [17]:
def evaluation(
    estimator, X_test, y_test, y_test_pred, y_test_pred_prob,
    best_train_score, df_metrics, col, vectorizer_name, classifier_name, scorig=scoring
):

    # Get metrics dict
    (
        metrics_dict, df_cv_score_recall,
        cv_train_scores, cv_test_scores,
        cv_train_recall, cv_test_recall,
        cv_train_explained_variance_recall, cv_test_explained_variance_recall,
        explained_variance, accuracy, balanced_accuracy, precision, recall,
        f1, mcc, fm, kappa, gmean, report, cm, cm_normalized,
        average_precision, roc_auc, auc, fpr, tpr, threshold, 
        loss, precision_pr, recall_pr, threshold_pr,
    ) = compute_metrics(
        estimator, X_test, y_test, y_test_pred, y_test_pred_prob,
        col, vectorizer_name, classifier_name
    )

    # Print metrics
    print('=' * 20)
    print('~' * 20)
    print(f' Testing Metrics for {str(col)} - {vectorizer_name} + {classifier_name}')
    print('~' * 20)
    print(f'Classification Report:\n {metrics_dict["Classification Report"]}')
    print('-' * 20)
    for metric_name, metric_value in metrics_dict.items():
        if 'Threshold' not in metric_name:
            with contextlib.suppress(TypeError, ValueError):
                metric_value = float(metric_value)
            if isinstance(metric_value, (int, float)):
                print(f'{metric_name}: {round(metric_value, 2)}')
            else:
                print(f'{metric_name}:\n{metric_value}')
            print('-' * 20)

        # Fill Table DF
        if isinstance(metric_value, float):
            df_metrics.loc[
                (classifier_name), (col, vectorizer_name, metric_name)
            ] = metric_value
        else:
            df_metrics.loc[
                (classifier_name), (col, vectorizer_name, metric_name)
            ] = str(metric_value)

    print('=' * 20)

    # Plot Metrics
    plot_metrics(
        estimator, X_test, y_test, y_test_pred, y_test_pred_prob,
        col, vectorizer_name, classifier_name,
    )

    return df_metrics, metrics_dict, df_cv_score_recall


In [18]:
# Save Model
def save_table(
    df_metrics,
    col, vectorizer_name, classifier_name,
    results_save_path=results_save_path,
    table_save_path=table_save_path,
    method=method, save_name=None,
    compression=None, protocol=None, 
    path_suffix=None, 
):
    if save_name is None:
        save_name = 'Estimators Table'
    if compression is None:
        compression = False
    if protocol is None:
        protocol = pickle.HIGHEST_PROTOCOL
    if path_suffix is None:
        path_suffix = f' - {str(col)} - {vectorizer_name} + {classifier_name} (Save_protocol={protocol}).pkl'

    # Save metrics df
    print(f'Saving fitted estimator and table for {vectorizer_name} + {classifier_name}.')
    df_metrics.to_csv(f'{table_save_path}{save_name}.csv')
    df_metrics.to_pickle(f'{table_save_path}{save_name}.pkl')
    df_metrics.to_excel(f'{table_save_path}{save_name}.xlsx')
    df_metrics.to_latex(f'{table_save_path}{save_name}.tex')
    df_metrics.to_markdown(f'{table_save_path}{save_name}.md')
    df_metrics.to_html(f'{table_save_path}{save_name}.html')

    print('Done saving fitted estimator and table!')


In [19]:
# Save Model
def get_completed_estimators(
    results_save_path=results_save_path, method=method, classifiers_pipe=classifiers_pipe,
    estimators_list=None, used_classifiers=None,
):
    if estimators_list is None:
        estimators_list = []
    if used_classifiers is None:
        used_classifiers = []

    for estimator_path in glob.glob(f'{results_save_path}{method} Estimator - *.pkl'):
        classifier_name = estimator_path.split(f'{results_save_path}{method} ')[1].split(' + ')[1].split(' (Save_protocol=')[0]
        used_classifiers.append(classifier_name)
        with open(estimator_path, 'rb') as f:
            estimators_list.append(joblib.load(f))

    assert set(classifiers_pipe.keys()) == set(used_classifiers), 'Not all classifiers were used!'

    return estimators_list


In [20]:
def comparison_plots(
    estimators_list, X_test, y_test, col,
    curves_dict=None, cmap=plt.cm.Blues
):

    curves_dict = {
        'ROC Curve': metrics.RocCurveDisplay,
        'Precision Recall Curve': metrics.PrecisionRecallDisplay,
        'Calibration Curve': metrics.CalibrationDisplay,
    }

    assert len(estimators_list) != 0

    for curve_name, curve_package in curves_dict.items():
        print('-' * 20)
        print(f'{str(col)} - {str(curve_name)}')
        fig, ax = plt.subplots()
        ax.set_title(f'{str(col)} - {str(curve_name)}')
        for estimator in estimators_list:
            curve = curve_package.from_estimator(
                estimator, X_test, y_test, pos_label=1, ax=ax, cmap=cmap,
                name=f'{estimator.steps[0][0]} + {estimator.steps[1][0]} + {estimator.steps[-1][0]}'
            )
        show_and_close_plots()

        # Save Plots
        print('Saving plots.')
        for image_save_format in ['eps', 'png', 'svg']:
            curve.figure_.savefig(
                f'{plot_save_path}{method} {str(col)} - All {str(curve_name)}s.{image_save_format}',
                format=image_save_format,
                dpi=3000, bbox_inches='tight'
            )


# Training

In [21]:
%%time
print('#'*40)
print('Starting!')
print('#'*40)

analysis_columns = ['Warmth', 'Competence']
text_col = 'Job Description spacy_sentencized'

# Get existing estimators
col_names_list, vectorizer_names_list, classifier_names_list, estimator_names_list = get_existing_files()

# Load Table DF
df_metrics = make_df_metrics(vectorizers_pipe, classifiers_pipe, list(metrics_dict.keys()), analysis_columns)

# Identify cols, vectorizers and classifiers
for estimators_file in tqdm.tqdm(glob.glob(f'{results_save_path}*.pkl')):
    if ' Estimator - ' in estimators_file and 'Fitted ' not in estimators_file:
        col = estimators_file.split(f'{method} Estimator - ')[-1].split(' - ')[0]
        vectorizer_name = estimators_file.split(f'{col} - ')[-1].split(' + ')[0]
        classifier_name = estimators_file.split(f'{vectorizer_name} + ')[-1].split(' (Save_protocol=')[0]
        protocol = int(estimators_file.split(f'{vectorizer_name} + ')[-1].split(' (Save_protocol=')[-1].split(').pkl')[0])

        print('~'*20)
        print(f'Loading data for {col} - {vectorizer_name} + {classifier_name}')
        print('~'*20)
        # Load X, y, search_cv, estimator
        (
            grid_search, searchcv,
            X_train, y_train, y_train_pred,
            X_test, y_test, y_test_pred, y_test_pred_prob,
            df_feature_importances, estimator,
        ) = load_Xy_search_cv_estimator(
            col, vectorizer_name, classifier_name, protocol
        )

        print('-'*20)
        print(f'{"="*30} EVALUATING DATASET OF LENGTH {len(X_train)+len(X_test)} ON {col.upper()} {"="*30}')

        # Identify and name steps in estimator
        vectorizer = estimator[0]
        vectorizer_params = vectorizer.get_params()
        vectorizer_name = vectorizer.__class__.__name__
        selector = estimator[1]
        selector_params = selector.get_params()
        selector_name = selector.__class__.__name__
        classifier = estimator[-1]
        classifier_params = classifier.get_params()
        classifier_name = classifier.__class__.__name__
        if col == 'Warmth':
            resampler = estimator[-2]
            resampler_params = resampler.get_params()
            resampler_name = resampler.__class__.__name__

        # Print results
        print('='*20)
        print(
            f'GridSearch - Best mean train score: M = {float(best_mean_train_score:=searchcv.cv_results_["mean_train_score"][best_index:=searchcv.best_index_]):.2f}, SD = {int(best_std_train_score:=searchcv.cv_results_["std_train_score"][best_index]):.2f}\n'
        )
        print(
            f'GridSearch - Best mean test score: M = {float(best_mean_test_score:=searchcv.cv_results_["mean_test_score"][best_index]):.2f}, SD = {int(best_std_test_score:=searchcv.cv_results_["std_test_score"][best_index]):.2f}\n'
        )
        print(
            f'Number of splits: {int(n_splits:=searchcv.n_splits_)}\n'
        )
        print(
            f'Best estimator and parameters:\n{estimator}\n')
        print(
            f'Best parameters:\n{(best_params:=searchcv.best_params_)}\n'
        )
        print(
            f'Training Classification Report:\n{(train_report:=metrics.classification_report(y_train, y_train_pred, labels=np.unique(y_train_pred), zero_division=0))}\n'
        )
        print(
            f'Best train score: {float(best_train_score:=searchcv.best_score_):.2f}\n'
        )
        print(
            f'Best test score: {float(best_test_score:=searchcv.score(X_test, y_test)):.2f}\n'
        )
        print('='*20)
        print('Training Confusion Matrix:\n')
        close_plots()
        fig, ax = plt.subplots()
        ax.set_title(
            f'{str(col)} - Training Confusion Matrix - {vectorizer_name} + {classifier_name}')
        train_cm = metrics.ConfusionMatrixDisplay.from_estimator(
            estimator, X_train, y_train, ax=ax, cmap=plt.cm.Blues
        )
        show_and_close_plots()
        print('='*20)

        # Evaluate Model
        df_metrics, metrics_dict, df_cv_score_recall = evaluation(
            estimator, X_test, y_test, y_test_pred, y_test_pred_prob,
            best_train_score, df_metrics,
            col, vectorizer_name, classifier_name, 
        )
        sys.exit(0)

        # # Fit best model on validation set
        # print(f'Fitting {estimator}.')
        # estimator.set_params(**estimator.get_params())
        # estimator = estimator.fit(X_val, y_val)

        # Save Vectorizer, Selector, and Classifier
        save_table(df_metrics, col, vectorizer_name, classifier_name)

# Compare Estimators
print('='*20)
print(f'Comparing Estimators for {col}')
comparison_plots(get_completed_estimators(), X_test, y_test, col)
print('='*20)

print('#'*40)
print('DONE!')
print('#'*40)


########################################
Starting!
########################################
Searching for existing estimators in directory:
/Users/nyxinsane/Documents/Work - UvA/Automating Equity/Study 1/Study1_Code/data/classification models/Supervised Results/


  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

~~~~~~~~~~~~~~~~~~~~
Loading data for Warmth - CountVectorizer + KNeighborsClassifier
~~~~~~~~~~~~~~~~~~~~
Done loading Xy, CV data, and estimator!
['df_cv_results', 'Grid Search', 'df_train_data', 'df_test_data', 'SearchCV', 'Estimator']
Done splitting data into training and testing sets.
Training set shape: (4483,)
----------
Training set example:
Factors)
~~~~~~~~~~
Testing set shape: (1495,)
----------
Testing set example:
Open proposals of candidates are at own risk.
~~~~~~~~~~
Training data class weights:
Ratio = 0.33 (0 = 0.67, 1 = 2.00)
----------
Testing data class weights:
Ratio = 0.32 (0 = 0.66, 1 = 2.06)
--------------------
============================== EVALUATING DATASET OF LENGTH 5978 ON WARMTH ==============================
GridSearch - Best mean train score: M = 0.68, SD = 0.00

GridSearch - Best mean test score: M = 0.67, SD = 0.00

Number of splits: 30

Best estimator and parameters:
Pipeline(steps=[('CountVectorizer',
                 CountVectorizer(lowercase=Fals

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
/opt/homebrew/Caskroom/mambaforge/base/envs/study1_3.10/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/opt/homebrew/Caskroom/mambaforge/base/envs/study1_3.10/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/opt/homebrew/Caskroom/mambaforge/base/envs/study1_3.10/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
[Parallel(n_jobs=-1)]: Done 150 out of 150 | 

...

Saved Normalized Confusion Matrix!
Plotting ROC Curve:
Saving ROC Curve...


Saved ROC Curve!
Plotting Precision-Recall Curve:
Saving Precision-Recall Curve...


Saved Precision-Recall Curve!
Plotting Calibration Curve:
Saving Calibration Curve...
Saved Calibration Curve!


  0%|          | 0/6 [03:11<?, ?it/s]
/opt/homebrew/Caskroom/mambaforge/base/envs/study1_3.10/lib/python3.10/site-packages/plot_metric/functions.py:169: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if function.__name__ is "plot_precision_recall_curve":
/opt/homebrew/Caskroom/mambaforge/base/envs/study1_3.10/lib/python3.10/site-packages/plot_metric/functions.py:171: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif function.__name__ is "plot_confusion_matrix":
/opt/homebrew/Caskroom/mambaforge/base/envs/study1_3.10/lib/python3.10/site-packages/plot_metric/functions.py:173: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif function.__name__ is "plot_roc_curve":
/opt/homebrew/Caskroom/mambaforge/base/envs/study1_3.10/lib/python3.10/site-packages/plot_metric/functions.py:175: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif function.__name__ is "plot_class_distribution":
/opt/homebrew/Caskroom/mambaforge/base/envs/study1_3.10/lib/python3.10/

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /opt/homebrew/Caskroom/mambaforge/base/envs/study1_3.10/lib/python3.10/site-packages/IPython/cor │
│ e/magics/execution.py:1319 in time                                                               │
│                                                                                                  │
│   1316 │   │   else:                                                                             │
│   1317 │   │   │   st = clock2()                                                                 │
│   1318 │   │   │   try:                                                                          │
│ ❱ 1319 │   │   │   │   exec(code, glob, local_ns)                                                │
│   1320 │   │   │   │   out=None                                                                  │
│   1321 │   │   │   │   # multi-line %%time case                                                  │
│   1322 │   │   │   │   if expr_val is not None:                                                  │
│ <timed exec>:91 in <module>                                                                      │
│                                                                                                  │
│ /var/folders/46/q15p556n1dd63z6gkwyh896c0000gn/T/ipykernel_82537/1389752298.py:51 in evaluation  │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/var/folders/46/q15p556n1dd63z6gkwyh896c0000gn/T/ipykernel_82537/1389752298.py'                 │
│                                                                                                  │
│ /var/folders/46/q15p556n1dd63z6gkwyh896c0000gn/T/ipykernel_82537/2685343375.py:24 in             │
│ plot_metrics                                                                                     │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/var/folders/46/q15p556n1dd63z6gkwyh896c0000gn/T/ipykernel_82537/2685343375.py'                 │
│                                                                                                  │
│ /var/folders/46/q15p556n1dd63z6gkwyh896c0000gn/T/ipykernel_82537/1331889538.py:85 in             │
│ plot_metrics_with_y_pred                                                                         │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/var/folders/46/q15p556n1dd63z6gkwyh896c0000gn/T/ipykernel_82537/1331889538.py'                 │
│                                                                                                  │
│ /opt/homebrew/Caskroom/mambaforge/base/envs/study1_3.10/lib/python3.10/site-packages/plot_metric │
│ /functions.py:663 in plot_class_distribution                                                     │
│                                                                                                  │
│    660 │   │                                                                                     │
│    661 │   │   # Plot violin pred distribution                                                   │
│    662 │   │   if display_violin:                                                                │
│ ❱  663 │   │   │   sns.violinplot(x='class', y='pred', data=pred_df, inner=None, color=c_violin  │
│    664 │   │                                                                                     │
│    665 │   │   # Plot prediction distribution                                                    │
│    666 │   │   if display_prediction:                      